In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

def get_one_page(counts):#加载页面，counts为点击加载更多评论的次数
    driver=webdriver.PhantomJS(executable_path="C:\\Users\\pujing\\Anaconda3\\phantomjs.exe")
    driver.get("http://zhushou.360.cn/detail/index/soft_id/3260425?recrefer=SE_D_%E5%B9%B3%E5%AE%89%E5%8F%A3%E8%A2%8B%E9%93%B6%E8%A1%8C")

    #防止引用太多，不断click，直到参考文献下不存在‘加载更多’
    try:
        for i in range(0,counts):
            # 等待网站加载完成
            time.sleep(0.2)
            driver.find_elements_by_class_name('icon-1')[0].click()
    except:
        print('********************************************************')
    html=driver.page_source      #获取加载后的网页
    return html,counts

def parse_one_page_and_write_as_csv(html,counts,file_path):#解析一个页面并保存为csv文件
    res_pingfen=[]
    res_users=[]
    res_comments=[]
    res_times=[]
    soup=BeautifulSoup(html,'lxml')
    comment_lists=soup.find('ul',{'id':'review-panel'}).find_all('li')
    print("加载",counts,"次加载了",len(comment_lists),'条数据！')


    #获取一个页面的用户评分
    fen_ls = '<span.*?"scmt-result">(.*?)</span>'
    fen_pattern = re.compile(fen_ls, re.S)
    fen_lists = re.findall(fen_pattern, str(comment_lists))
    for li in fen_lists:
        #print(li)
        res_pingfen.append(li)

    #获取一个页面的用户名称
    user_ls = '<span.*?"scmt-usr">(.*?)</span>'
    user_pattern = re.compile(user_ls, re.S)
    user_lists = re.findall(user_pattern, str(comment_lists))
    for li in user_lists:
        #print(li)
        res_users.append(li)

    #获取一个页面的用户评论
    pinglun_ls = '<span.*?"scmt-usr">.*?</span>.*?<span.*?"word-break:break-all;">(.*?)</span>'
    pinglun_pattern = re.compile(pinglun_ls, re.S)
    pinglun_lists = re.findall(pinglun_pattern, str(comment_lists))
    for li in pinglun_lists:
        #print(li)
        res_comments.append(li)

    #获取一个页面的评论时间
    time_ls = '<p.*?"last">(.*?)</p>'
    time_pattern = re.compile(time_ls, re.S)
    time_lists = re.findall(time_pattern, str(comment_lists))
    for li in time_lists:
        #print(li)
        res_times.append(li)
        
    data=[res_users,res_pingfen,res_times,res_comments]
    result=pd.DataFrame(data)
    result=result.T
    result.columns=['用户名称','评分','时间','评论']
    result.to_csv(file_path+'360手机助手_平安口袋银行.csv')

def phone_helper_360(file_path):
    html,counts=get_one_page(300)
    parse_one_page_and_write_as_csv(html,counts,file_path)
        
if __name__=='__main__':
    file_path='C:\\Users\\pujing\\Desktop\\长亮科技\\'
    phone_helper_360(file_path)

加载 300 次加载了 1826 条数据！
